In [1]:
import requests
import statistics
import csv
from datetime import datetime, timedelta

# Capitales departamentales del Perú con coordenadas
departamentos = {
    "Amazonas": (-6.2290, -77.8721),        # Chachapoyas
    "Áncash": (-9.5278, -77.5288),          # Huaraz
    "Apurímac": (-13.6364, -72.8814),       # Abancay
    "Arequipa": (-16.4090, -71.5375),       # Arequipa
    "Ayacucho": (-13.1588, -74.2239),       # Ayacucho
    "Cajamarca": (-7.1618, -78.5128),       # Cajamarca
    "Cusco": (-13.5319, -71.9675),          # Cusco
    "Huancavelica": (-12.7860, -74.9769),   # Huancavelica
    "Huánuco": (-9.9306, -76.2422),         # Huánuco
    "Ica": (-14.0678, -75.7286),            # Ica
    "Junín": (-11.4192, -75.6900),          # Tarma (reemplazar con Huancayo si se quiere exacto)
    "La Libertad": (-8.1150, -79.0299),     # Trujillo
    "Lambayeque": (-6.7714, -79.8409),      # Chiclayo
    "Lima": (-12.0464, -77.0428),           # Lima
    "Loreto": (-3.7437, -73.2516),          # Iquitos
    "Madre de Dios": (-12.5933, -69.1890),  # Puerto Maldonado
    "Moquegua": (-17.1942, -70.9352),       # Moquegua
    "Pasco": (-10.6820, -76.2560),          # Cerro de Pasco
    "Piura": (-5.1945, -80.6328),           # Piura
    "Puno": (-15.8402, -70.0219),           # Puno
    "San Martín": (-6.0343, -76.9714),      # Moyobamba
    "Tacna": (-18.0146, -70.2538),          # Tacna
    "Tumbes": (-3.5669, -80.4515),          # Tumbes
    "Ucayali": (-8.3924, -74.5826),         # Pucallpa
}

# Meses del año con número de días estándar (ajustado con timedelta después)
meses = {
    1: "Ene", 2: "Feb", 3: "Mar", 4: "Abr", 5: "May", 6: "Jun",
    7: "Jul", 8: "Ago", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dic"
}

# Años de interés
anios = [2020, 2021, 2022, 2023, 2024]

# Lista para almacenar resultados
resultados = []

for dpto, (lat, lon) in departamentos.items():
    fila = {"Departamento": dpto}

    for anio in anios:
        for mes_num in range(1, 13):
            # Fecha de inicio
            start_date = datetime(anio, mes_num, 1)
            # Fecha de fin = inicio del siguiente mes - 1 día
            if mes_num == 12:
                end_date = datetime(anio + 1, 1, 1) - timedelta(days=1)
            else:
                end_date = datetime(anio, mes_num + 1, 1) - timedelta(days=1)

            # Formatear fechas
            start_str = start_date.strftime("%Y-%m-%d")
            end_str = end_date.strftime("%Y-%m-%d")
            col_label = f"{meses[mes_num]}-{anio}"

            url = (
                "https://archive-api.open-meteo.com/v1/archive"
                f"?latitude={lat}&longitude={lon}"
                f"&start_date={start_str}&end_date={end_str}"
                "&daily=temperature_2m_mean"
                "&timezone=America/Lima"
            )

            response = requests.get(url)
            if response.status_code == 200:
                data = response.json()
                temps = data.get("daily", {}).get("temperature_2m_mean", [])
                if temps:
                    promedio = round(statistics.mean(temps), 2)
                    fila[col_label] = promedio
                else:
                    fila[col_label] = "ND"
            else:
                fila[col_label] = "Error"

    resultados.append(fila)

# Ordenar columnas
columnas = ["Departamento"] + [
    f"{meses[m]}-{a}" for a in anios for m in range(1, 13)
]

# Guardar en CSV
csv_file = "temperatura_promedio_2020_2024.csv"
with open(csv_file, mode='w', newline='', encoding='utf-8-sig') as file:
    writer = csv.DictWriter(file, fieldnames=columnas)
    writer.writeheader()
    writer.writerows(resultados)

print(f"\n✅ Archivo CSV generado: {csv_file}")
print("Contiene temperatura promedio mensual de 2020 a 2024 por departamento.")




✅ Archivo CSV generado: temperatura_promedio_2020_2024.csv
Contiene temperatura promedio mensual de 2020 a 2024 por departamento.
